In [1]:
import pandas as pd
import numpy as np


from data_loader import load_from_tsfile_to_dataframe
from regressor_tools import process_data
import mlflow
from tsfeatures import tsfeatures

In [ ]:
# DCT or DFT Compression
# precise cutting and not with threshold_value

#->> both methods should be alright, but need to be field tested!

import numpy as np
from scipy.fft import dct, idct, fft, ifft

def dct_or_dft_compress(signal, dropout_ratio, transform_type='dft'):
    if transform_type == 'dct':
        # Apply DCT to the signal
        signal_transformed = dct(signal, type=2, norm='ortho')
        inverse_transform = idct
    elif transform_type == 'dft':
        # Apply FFT to the signal
        signal_transformed = fft(signal)
        inverse_transform = ifft
    else:
        raise ValueError("Invalid transform_type. Choose 'dct' or 'dft'.")
    
    # Calculate the number of coefficients to keep
    num_coeffs = int((1 - dropout_ratio) * len(signal_transformed))
    
    # Sort the coefficients by magnitude and keep only the top ones
    sorted_indices = np.argsort(np.abs(signal_transformed))
    kept_indices = sorted_indices[-num_coeffs:]
    
    # Set the coefficients to zero for the dropped out indices
    compressed_transform = np.zeros_like(signal_transformed)
    compressed_transform[kept_indices] = signal_transformed[kept_indices]
    
    # Reconstruct the signal using inverse transform
    compressed_signal = inverse_transform(compressed_transform)
    
    return compressed_signal

In [4]:
# Test dimension flattening for compresssion
import numpy as np

# Define the array dimensions
num_dp = 6
len_ts = 3
dim = 4

# Create an example array with shape (6, 3, 4)
data = np.array([
    [
        [1, 2, 3, 4],
        [5, 6, 7, 8],
        [9, 10, 11, 12]
    ],
    [
        [13, 14, 15, 16],
        [17, 18, 19, 20],
        [21, 22, 23, 24]
    ],
    [
        [25, 26, 27, 28],
        [29, 30, 31, 32],
        [33, 34, 35, 36]
    ],
    [
        [37, 38, 39, 40],
        [41, 42, 43, 44],
        [45, 46, 47, 48]
    ],
    [
        [49, 50, 51, 52],
        [53, 54, 55, 56],
        [57, 58, 59, 60]
    ],
    [
        [61, 62, 63, 64],
        [65, 66, 67, 68],
        [69, 70, 71, 72]
    ]
])

print("Original array:")
print(data)
print("Original shape:", data.shape)
# Output: (6, 3, 4)

# Reshape the array to (num_dp * len_ts, dim)
flattened_data = data.reshape(-1, dim)

print("\nFlattened array:")
print(flattened_data)
print("New shape:", flattened_data.shape)
# Output: (18, 4)


Original array:
[[[ 1  2  3  4]
  [ 5  6  7  8]
  [ 9 10 11 12]]

 [[13 14 15 16]
  [17 18 19 20]
  [21 22 23 24]]

 [[25 26 27 28]
  [29 30 31 32]
  [33 34 35 36]]

 [[37 38 39 40]
  [41 42 43 44]
  [45 46 47 48]]

 [[49 50 51 52]
  [53 54 55 56]
  [57 58 59 60]]

 [[61 62 63 64]
  [65 66 67 68]
  [69 70 71 72]]]
Original shape: (6, 3, 4)

Flattened array:
[[ 1  2  3  4]
 [ 5  6  7  8]
 [ 9 10 11 12]
 [13 14 15 16]
 [17 18 19 20]
 [21 22 23 24]
 [25 26 27 28]
 [29 30 31 32]
 [33 34 35 36]
 [37 38 39 40]
 [41 42 43 44]
 [45 46 47 48]
 [49 50 51 52]
 [53 54 55 56]
 [57 58 59 60]
 [61 62 63 64]
 [65 66 67 68]
 [69 70 71 72]]
New shape: (18, 4)


In [7]:
import numpy as np
from numpy.polynomial.chebyshev import chebfit, chebval


# Chebyshev Transoform Compression Code -WIP
def cpt_compress(signal, dropout_ratio, andDecompress:bool):
    
    # Later: Try with fitting into Interval -1,1

    # Evaluation points
    t = np.arange(signal.size)

    # Get the coefficients. deg is the longest that is possible. Shouldn't take too long to calculate.
    cpt_coeffs = chebfit(t, signal, deg=len(signal)-1)

    # Calculate the number of coefficients to zero out
    num_coeffs = int((dropout_ratio) * len(cpt_coeffs))
    
    # Sort the coefficients by value and cut off the smallest ones
    sorted_indices = np.argsort(np.abs(cpt_coeffs))
    indices_to_zero = sorted_indices[:num_coeffs]

    # Zero out selected coefficients
    cpt_coeffs[indices_to_zero] = 0

    if andDecompress == False:
        return cpt_coeffs
    else:
        decompressed_signal = chebval(t, cpt_coeffs)
        return decompressed_signal


In [9]:
cpt_compress(np.array(), 0.9, True)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])